In [ ]:
# # ----------------------------- Show All rows & columns -----------------------------

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# # ----------------------------- play_sound -----------------------------

# import winsound

# def play_sound(sound, Symbol_error, error_true):

#     # Alert_01, Alert_02, Atma_rama_Alarm
#     winsound.PlaySound(r"D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\{}.wav".format(sound), winsound.SND_FILENAME | winsound.SND_ASYNC)
#     print(f"\033[1;31m{Symbol_error}\033[0m")
        
#     if error_true == "error":
#         raise Exception(f"\033[1;31m{Symbol_error}\033[0m")

# play_sound("Atma_rama_Alarm", "Symbol error", "error")     # Alert_01, Alert_02, Atma_rama_Alarm

In [ ]:
# # ----------------------------- ZeroLossMarketFeed Raw code "Cell: 1" -----------------------------

# # Enable nested event loops for Jupyter
# import nest_asyncio
# nest_asyncio.apply()

# import asyncio
# import ast
# from dhanhq import dhanhq, DhanContext, MarketFeed
# import functions as f
# import pandas as pd
# import numpy as np
# from collections import deque
# import time
# import threading

# class ZeroLossMarketFeed:
#     def __init__(self, dhan_context, instruments, max_instruments_per_conn=50):
#         self.dhan_context = dhan_context
#         self.max_instruments_per_conn = max_instruments_per_conn
#         self.tick_buffer = deque(maxlen=100000)  # Large buffer
#         self.is_running = False
#         self.connections = []
        
#         # Split instruments into batches
#         self.instrument_batches = self._split_instruments(instruments)
        
#     def _split_instruments(self, instruments):
#         """Split instruments into batches of max_instruments_per_conn"""
#         batches = []
#         for i in range(0, len(instruments), self.max_instruments_per_conn):
#             batch = instruments[i:i + self.max_instruments_per_conn]
#             batches.append(batch)
#         return batches
    
#     async def _handle_connection(self, instruments_batch, batch_id):
#         """Handle a single connection for a batch of instruments"""
#         while self.is_running:
#             try:
#                 print(f"Starting connection {batch_id} with {len(instruments_batch)} instruments")
                
#                 # Create connection
#                 feed = MarketFeed(self.dhan_context, instruments_batch, "v2")
#                 await feed.connect()
                
#                 # Continuous data reception
#                 while self.is_running:
#                     try:
#                         response = await feed.get_instrument_data()
#                         if response:
#                             # Immediately buffer the tick
#                             self.tick_buffer.append({
#                                 'timestamp': time.time(),
#                                 'batch_id': batch_id,
#                                 'data': response
#                             })
                            
#                             # Process tick immediately (non-blocking)
#                             self._process_tick_nonblocking(response)
                            
#                     except Exception as e:
#                         print(f"Data reception error in batch {batch_id}: {e}")
#                         await asyncio.sleep(0.1)
#                         break
                        
#             except Exception as e:
#                 print(f"Connection error in batch {batch_id}: {e}", "\tCheck Internet connection")
#                 await asyncio.sleep(1)  # Wait before reconnect
    
#     def _process_tick_nonblocking(self, tick_data):
#         """Process tick data without blocking the main loop"""
#         try:
#             # Your strategy logic here - keep it fast!
#             # Example: Just print security_id and LTP
#             if isinstance(tick_data, dict):
#                 print(tick_data)
                
#         except Exception as e:
#             print(f"Tick processing error: {e}")
    
#     async def start_all_connections(self):
#         """Start all connections concurrently"""
#         self.is_running = True
        
#         # Create tasks for each batch
#         tasks = []
#         for i, batch in enumerate(self.instrument_batches):
#             task = asyncio.create_task(
#                 self._handle_connection(batch, f"batch_{i}")
#             )
#             tasks.append(task)
        
#         print(f"Starting {len(tasks)} connections...")
        
#         # Run all connections concurrently
#         try:
#             await asyncio.gather(*tasks)
#         except KeyboardInterrupt:
#             print("Stopping all connections...")
#             self.is_running = False
            
#     def stop(self):
#         """Stop all connections"""
#         self.is_running = False
        
#     def get_buffer_stats(self):
#         """Get buffer statistics"""
#         return {
#             'buffer_size': len(self.tick_buffer),
#             'max_buffer': self.tick_buffer.maxlen,
#             'buffer_usage': f"{len(self.tick_buffer)/self.tick_buffer.maxlen*100:.1f}%"
#         }

# # Your existing setup code
# client_id = str(ast.literal_eval(f.get_line(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\database.txt', 3).strip())['client_id'])
# access_token = str(ast.literal_eval(f.get_line(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\database.txt', 4).strip())['access_token'])

# dhan_context = DhanContext(client_id, access_token)
# dhan = dhanhq(dhan_context)

# # Load and filter your option chain data
# option_chain_df = pd.read_csv(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\Option_Chain_with_Security_id.csv')

# def get_prices_under_1000(df):
#     first_ce_idx = df[df['ce_last_price'] >= 800].index.max()
#     first_pe_idx = df[df['pe_last_price'] >= 800].index.min()
#     filtered_df = df[first_ce_idx:first_pe_idx].reset_index(drop=True)
    
#     ce_cols = ['security_id_CE', 'ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price']
#     pe_cols = ['pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume', 'security_id_PE']
    
#     filtered_df.loc[filtered_df['ce_last_price'] < 10, ce_cols] = 0.0
#     filtered_df.loc[filtered_df['pe_last_price'] < 10, pe_cols] = 0.0
    
#     return filtered_df

# def create_instruments_from_df(df):
#     instruments = []
#     for _, row in df.iterrows():
#         if pd.notna(row['security_id_CE']) and row['security_id_CE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_CE'])), MarketFeed.Ticker))
#         if pd.notna(row['security_id_PE']) and row['security_id_PE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_PE'])), MarketFeed.Ticker))
#     return instruments

# # Setup instruments
# filtered_df = get_prices_under_1000(option_chain_df)
# instruments = create_instruments_from_df(filtered_df)
# instruments = instruments[:1]

# print(f"Total instruments: {len(instruments)}")

# # MAIN EXECUTION
# async def main():
#     # Create zero-loss feed with max 40 instruments per connection (safer than 50)
#     feed_manager = ZeroLossMarketFeed(dhan_context, instruments, max_instruments_per_conn=40)
    
#     try:
#         # Start all connections
#         await feed_manager.start_all_connections()
        
#     except KeyboardInterrupt:
#         print("Stopping feed...")
#         feed_manager.stop()
    
#     except Exception as e:
#         print(f"Feed error: {e}")
#         feed_manager.stop()

# # Run the async main function
# if __name__ == "__main__":
#     try:
#         asyncio.run(main())
#     except KeyboardInterrupt:
#         print("Feed stopped by user")
#     except Exception as e:
#         print(f"Fatal error: {e}")

In [ ]:
# # ----------------------------- ZeroLossMarketFeed Raw code "Cell: 2" -----------------------------

# # Enable nested event loops for Jupyter
# import nest_asyncio
# nest_asyncio.apply()

# import asyncio
# import ast
# from dhanhq import dhanhq, DhanContext, MarketFeed
# import functions as f
# import pandas as pd
# import numpy as np
# from collections import deque
# import time
# import threading

# class ZeroLossMarketFeed:
#     def __init__(self, dhan_context, instruments, max_instruments_per_conn=50):
#         self.dhan_context = dhan_context
#         self.max_instruments_per_conn = max_instruments_per_conn
#         self.tick_buffer = deque(maxlen=100000)  # Large buffer
#         self.is_running = False
#         self.connections = []
        
#         # Split instruments into batches
#         self.instrument_batches = self._split_instruments(instruments)
        
#     def _split_instruments(self, instruments):
#         """Split instruments into batches of max_instruments_per_conn"""
#         batches = []
#         for i in range(0, len(instruments), self.max_instruments_per_conn):
#             batch = instruments[i:i + self.max_instruments_per_conn]
#             batches.append(batch)
#         return batches
    
#     async def _handle_connection(self, instruments_batch, batch_id):
#         """Handle a single connection for a batch of instruments"""
#         while self.is_running:
#             try:
#                 print(f"Starting connection {batch_id} with {len(instruments_batch)} instruments")
                
#                 # Create connection
#                 feed = MarketFeed(self.dhan_context, instruments_batch, "v2")
#                 await feed.connect()
                
#                 # Continuous data reception
#                 while self.is_running:
#                     try:
#                         response = await feed.get_instrument_data()
#                         if response:
#                             # Immediately buffer the tick
#                             self.tick_buffer.append({
#                                 'timestamp': time.time(),
#                                 'batch_id': batch_id,
#                                 'data': response
#                             })
                            
#                             # Process tick immediately (non-blocking)
#                             self._process_tick_nonblocking(response)
                            
#                     except Exception as e:
#                         print(f"Data reception error in batch {batch_id}: {e}")
#                         await asyncio.sleep(0.1)
#                         break
                        
#             except Exception as e:
#                 print(f"Connection error in batch {batch_id}: {e}", "\tCheck Internet connection")
#                 await asyncio.sleep(1)  # Wait before reconnect
    
#     def _process_tick_nonblocking(self, tick_data):
#         """Process tick data without blocking the main loop"""
#         try:
#             # Your strategy logic here - keep it fast!
#             # Example: Just print security_id and LTP
#             if isinstance(tick_data, dict):
#                 print(tick_data)
                
#         except Exception as e:
#             print(f"Tick processing error: {e}")
    
#     async def start_all_connections(self):
#         """Start all connections concurrently"""
#         self.is_running = True
        
#         # Create tasks for each batch
#         tasks = []
#         for i, batch in enumerate(self.instrument_batches):
#             task = asyncio.create_task(
#                 self._handle_connection(batch, f"batch_{i}")
#             )
#             tasks.append(task)
        
#         print(f"Starting {len(tasks)} connections...")
        
#         # Run all connections concurrently
#         try:
#             await asyncio.gather(*tasks)
#         except KeyboardInterrupt:
#             print("Stopping all connections...")
#             self.is_running = False
            
#     def stop(self):
#         """Stop all connections"""
#         self.is_running = False
        
#     def get_buffer_stats(self):
#         """Get buffer statistics"""
#         return {
#             'buffer_size': len(self.tick_buffer),
#             'max_buffer': self.tick_buffer.maxlen,
#             'buffer_usage': f"{len(self.tick_buffer)/self.tick_buffer.maxlen*100:.1f}%"
#         }

# # Your existing setup code
# client_id = str(ast.literal_eval(f.get_line(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\database.txt', 3).strip())['client_id'])
# access_token = str(ast.literal_eval(f.get_line(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\database.txt', 4).strip())['access_token'])

# dhan_context = DhanContext(client_id, access_token)
# dhan = dhanhq(dhan_context)

# # Load and filter your option chain data
# option_chain_df = pd.read_csv(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\Option_Chain_with_Security_id.csv')

# def get_prices_under_1000(df):
#     first_ce_idx = df[df['ce_last_price'] >= 800].index.max()
#     first_pe_idx = df[df['pe_last_price'] >= 800].index.min()
#     filtered_df = df[first_ce_idx:first_pe_idx].reset_index(drop=True)
    
#     ce_cols = ['security_id_CE', 'ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price']
#     pe_cols = ['pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume', 'security_id_PE']
    
#     filtered_df.loc[filtered_df['ce_last_price'] < 10, ce_cols] = 0.0
#     filtered_df.loc[filtered_df['pe_last_price'] < 10, pe_cols] = 0.0
    
#     return filtered_df

# def create_instruments_from_df(df):
#     instruments = []
#     for _, row in df.iterrows():
#         if pd.notna(row['security_id_CE']) and row['security_id_CE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_CE'])), MarketFeed.Ticker))
#         if pd.notna(row['security_id_PE']) and row['security_id_PE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_PE'])), MarketFeed.Ticker))
#     return instruments

# # Setup instruments
# filtered_df = get_prices_under_1000(option_chain_df)
# instruments = create_instruments_from_df(filtered_df)
# # instruments = instruments[26:]
# # instruments = instruments[:1]
# instruments

# print(f"Total instruments: {len(instruments)}")

# # MAIN EXECUTION
# async def main():
#     # Create zero-loss feed with max 40 instruments per connection (safer than 50)
#     feed_manager = ZeroLossMarketFeed(dhan_context, instruments, max_instruments_per_conn=40)
    
#     try:
#         # Start all connections
#         await feed_manager.start_all_connections()
        
#     except KeyboardInterrupt:
#         print("Stopping feed...")
#         feed_manager.stop()
    
#     except Exception as e:
#         print(f"Feed error: {e}")
#         feed_manager.stop()

# # Run the async main function
# if __name__ == "__main__":
#     try:
#         asyncio.run(main())
#     except KeyboardInterrupt:
#         print("Feed stopped by user")
#     except Exception as e:
#         print(f"Fatal error: {e}")

In [ ]:
# # ----------------------------- Get 1 minute data with retry -----------------------------

# import datetime
# import pandas as pd
# import time

# def get_data(security_id):
    
#     today = datetime.today().strftime("%Y-%m-%d")

#     for i in range(0, 3):
#         # Get the latest minute chart data
#         minute_chart = dhan.intraday_minute_data(
#             security_id=security_id,
#             exchange_segment="NSE_FNO", 
#             instrument_type="OPTIDX",
#             from_date=today,
#             to_date=today, 
#             interval=1
#         )

#         # Check if data is None or empty
#         if not minute_chart["data"]:
#             print("Your API subscription has expired or API related error", security_id)
#             continue

#         # Convert the data to a DataFrame
#         minute_chart_df = pd.DataFrame(minute_chart["data"])
#         minute_chart_df['datetime'] = pd.to_datetime(minute_chart_df['timestamp'], unit='s') + pd.Timedelta(hours=5, minutes=30)
#         minute_chart_df.drop(columns=['timestamp'], inplace=True)

#         if minute_chart_df is not None:
#             return minute_chart_df
        
#         time.sleep(0.14)
        

In [ ]:
# # ----------------------------- Get index of price filter effeciently -----------------------------

# def fast_select_three_rows(df, price_col):
#     # Convert to NumPy array for speed
#     prices = df[price_col].values
#     idx_below = (prices < 270).nonzero()[0]
    
#     if len(idx_below) == 0:
#         raise ValueError(f"No {price_col} < 270 found.")
    
#     # Get index of max price below 270
#     middle_pos = idx_below[prices[idx_below].argmax()]

#     # Collect surrounding indices
#     start = max(0, middle_pos - 1)
#     end = min(len(df), middle_pos + 2)

#     return df.iloc[start:end]

# # Usage:
# ce_selected = fast_select_three_rows(df, 'ce_last_price').copy()
# ce_selected.drop(columns=['pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume', 'security_id_PE'], inplace=True)

# pe_selected = fast_select_three_rows(df, 'pe_last_price').copy()
# pe_selected.drop(columns=['security_id_CE', 'ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price'], inplace=True)

In [ ]:
# # ----------------------------- Tick data count analysis for market feed "Cell: 1" -----------------------------

# import os
# import pandas as pd
# import struct
# from pathlib import Path
# from datetime import datetime
# from dhanhq import MarketFeed

# class TickFileReader:
#     """Fast reader for tick files - supports parallel reading"""
    
#     RECORD_SIZE = 32
    
#     @staticmethod
#     def read_last_tick(filepath):
#         """Read only the last tick - ultra fast"""
#         try:
#             with open(filepath, 'rb') as f:
#                 # Seek to last complete record
#                 f.seek(-TickFileReader.RECORD_SIZE, 2)
#                 data = f.read(TickFileReader.RECORD_SIZE)
                
#                 if len(data) == TickFileReader.RECORD_SIZE:
#                     time_bytes, price, oi, volume = struct.unpack('8sddd', data)
#                     ltt = time_bytes.rstrip(b'\x00').decode('utf-8')
#                     return {
#                         'LTT': ltt,
#                         'price': price,
#                         'OI': oi,
#                         'volume': volume
#                     }
#         except Exception as e:
#             print(f"❌ Error reading last tick: {e}")
#         return None
    
#     @staticmethod
#     def read_full_file(filepath):
#         """Read entire file - returns all ticks"""
#         ticks = []
#         try:
#             with open(filepath, 'rb') as f:
#                 while True:
#                     data = f.read(TickFileReader.RECORD_SIZE)
#                     if len(data) != TickFileReader.RECORD_SIZE:
#                         break
                    
#                     time_bytes, price, oi, volume = struct.unpack('8sddd', data)
#                     ltt = time_bytes.rstrip(b'\x00').decode('utf-8')
                    
#                     # Skip empty records
#                     if ltt:
#                         ticks.append({
#                             'LTT': ltt,
#                             'price': price,
#                             'OI': oi,
#                             'volume': volume
#                         })
#         except Exception as e:
#             print(f"❌ Error reading full file: {e}")
        
#         return ticks
    
# # PARALLEL READING FUNCTIONS
# def read_latest_tick(security_id, data_path=r"D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy"):
#     """Read latest tick for a security - ULTRA FAST"""
#     today_date = datetime.now().strftime('%Y-%m-%d')
#     filepath = Path(data_path) / "Tick_data" / f"market_data_{today_date}" / f"{security_id}.bin"
#     return TickFileReader.read_last_tick(filepath)

# def read_all_ticks(security_id, data_path=r"D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy"):
#     """Read all ticks for a security"""
#     today_date = datetime.now().strftime('%Y-%m-%d')
#     filepath = Path(data_path) / "Tick_data" / f"market_data_{today_date}" / f"{security_id}.bin"
#     return TickFileReader.read_full_file(filepath)


In [ ]:
# # ----------------------------- Tick data count analysis for market feed "Cell: 2" -----------------------------

# market_feed_data = [{'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 50993, 'LTP': '366.35', 'LTT': '12:11:41'},
#     {'type': 'Previous Close', 'exchange_segment': 2, 'security_id': 50993, 'prev_close': '226.85', 'prev_OI': 1236905968},
#     {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 50990, 'LTP': '37.40', 'LTT': '12:11:42'},
#     {'type': 'Previous Close', 'exchange_segment': 2, 'security_id': 50990, 'prev_close': '113.90', 'prev_OI': 1251054434},
#     {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 50961, 'LTP': '822.50', 'LTT': '12:08:45'},
#     {'type': 'Previous Close', 'exchange_segment': 2, 'security_id': 50961, 'prev_close': '622.35', 'prev_OI': 1198364928},
#     {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 51008, 'LTP': '136.75', 'LTT': '12:11:42'},
#     {'type': 'Previous Close', 'exchange_segment': 2, 'security_id': 51008, 'prev_close': '302.45', 'prev_OI': 1234819168},
#     {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 50963, 'LTP': '771.05', 'LTT': '12:11:13'}]


In [ ]:
# # ----------------------------- Tick data count analysis for market feed "Cell: 3" -----------------------------

# option_chain_df = pd.read_csv(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\data\Option_Chain_with_Security_id.csv')

# def get_prices_under_1000(df):
#     """Filter function exactly like original"""
#     first_ce_idx = df[df['ce_last_price'] >= 800].index.max()
#     first_pe_idx = df[df['pe_last_price'] >= 800].index.min()
#     filtered_df = df[first_ce_idx:first_pe_idx].reset_index(drop=True)
    
#     ce_cols = ['security_id_CE', 'ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price']
#     pe_cols = ['pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume', 'security_id_PE']
    
#     filtered_df.loc[filtered_df['ce_last_price'] < 10, ce_cols] = 0.0
#     filtered_df.loc[filtered_df['pe_last_price'] < 10, pe_cols] = 0.0
    
#     return filtered_df

# def create_instruments_from_df(df):
#     """Create instruments exactly like original"""
#     instruments = []
#     for _, row in df.iterrows():
#         if pd.notna(row['security_id_CE']) and row['security_id_CE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_CE'])), MarketFeed.Ticker))
#         if pd.notna(row['security_id_PE']) and row['security_id_PE'] != 0.0:
#             instruments.append((MarketFeed.NSE_FNO, str(int(row['security_id_PE'])), MarketFeed.Ticker))
#     return instruments

# # Setup instruments
# filtered_df = get_prices_under_1000(option_chain_df)
# instruments = create_instruments_from_df(filtered_df)

# # -----------------------------------------

# from_where_to_read = "raw_output"  # "raw_output" "from_file"

# if from_where_to_read == "raw_output":
    
#     market_feed_data_df = pd.DataFrame(market_feed_data)

#     security_id_counts = market_feed_data_df['security_id'].value_counts()

# if from_where_to_read == "from_file":

#     # Initialize an empty list to store all ticks
#     all_ticks = []

#     # Iterate over all files in the directory
#     for file_name in os.listdir(r'D:\Programming\Download_Backtest_Deploy_data\5__Deploy\Multi_deploy\Tick_data\market_data_2025-06-19'):
#         if file_name.endswith(".bin"):
#             # Extract the security_id from the file name
#             security_id = file_name.split(".")[0]
#             # Read all ticks for the security_id
#             ticks = read_all_ticks(security_id)
#             all_ticks.extend([int(security_id)] * int(len(ticks)))

#     # Create a DataFrame from the ticks and calculate security_id_counts
#     all_ticks_df = pd.DataFrame(all_ticks, columns=["security_id"])
#     security_id_counts = all_ticks_df['security_id'].value_counts()

# # -----------------------------------------

# # Create the count columns
# filtered_df["counts_ce"] = filtered_df["security_id_CE"].map(security_id_counts).fillna(0).astype(int)
# filtered_df["counts_pe"] = filtered_df["security_id_PE"].map(security_id_counts).fillna(0).astype(int)

# # Reorder columns
# cols = filtered_df.columns.tolist()

# # Move ce count to first, pe count to last
# cols.insert(0, cols.pop(cols.index("counts_ce")))  # move to front
# cols.append(cols.pop(cols.index("counts_pe")))     # move to end

# # Apply the new order
# filtered_df = filtered_df[cols]

# print(len(instruments))
# filtered_df

In [ ]:
# # ----------------------------- Check Internet Connection -----------------------------

# from datetime import datetime, timedelta
# import socket
# from IPython.display import display, HTML, Javascript
# import time

# def update_element(s):
#     display(HTML("""<div><pre id="clock" style='margin:0; padding:0;'></pre></div>"""))
#     display(Javascript(f"""document.getElementById('clock').innerHTML = '{s}';"""))

# def is_internet_available():

#     starting_time = str(datetime.now())
#     start_time = datetime.strptime(starting_time, "%Y-%m-%d %H:%M:%S.%f")

#     time_elapsed = None

#     while True:
#         try:
#             # Attempt to connect to Google's DNS server
#             socket.create_connection(("8.8.8.8", 53), timeout=3)
#             if time_elapsed is not None:
#                 print(f"Successfully connected to the internet after 0{time_elapsed}")
#             print(f"Internet Connection: 'Available'")
#             return True
#         except OSError:
            
#             current_time = datetime.now()
#             time_elapsed = str(current_time - start_time).split('.')[0]

#             update_element(f"Internet connection lost, retrying... Time Elapsed: 0{time_elapsed}")

#             time.sleep(1)

# is_internet_available()

In [ ]:
# pc_status_server.py
from http.server import BaseHTTPRequestHandler, HTTPServer

class StatusHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"PC is online")

def run():
    server_address = ('0.0.0.0', 8989)
    httpd = HTTPServer(server_address, StatusHandler)
    print("PC status server running on port 8989")
    httpd.serve_forever()

if __name__ == '__main__':
    run()


In [ ]:
# mobile_checker.py
import time
import requests
import os

PC_IP = "192.168.X.X"  # Replace with PC's local IP (for hotspot)
PC_PORT = 8989

def play_alarm():
    os.system('termux-media-player play /sdcard/alarm.mp3')

while True:
    try:
        r = requests.get(f"http://{PC_IP}:{PC_PORT}", timeout=3)
        print("PC is online:", r.text)
    except:
        print("PC is unreachable — triggering alarm")
        play_alarm()
        break  # Optional: break or keep trying
    time.sleep(5)
